In [2]:
import requests
import json
import psycopg2

url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json"
headers = {
    "accept": "application/json",
    "x-api-key": "lCd20Zc6VmbsEyIiMlPso3VOaMzgFxB3wC8k8s3Q"
}
response = requests.get(url, headers=headers)
data = json.loads(response.text)

com_list = []
category_list = []  

for com in data['competitions']:
    com_list.append({
        'competition_id': com['id'],
        'competition_name': com['name'],
        'type': com.get('type'),
        'gender': com.get('gender'),
        'parent_id': com.get('parent_id'),  # This safely assigns None if key is missing or null
        'category_id': com['category']['id']
    })

    category_list.append({
        'category_id': com['category']['id'],
        'category_name': com['category']['name']
    })


# Optional: remove duplicates from category list

# Connect to DB
conn = psycopg2.connect(
    host="localhost",
    database="sports_radar",
    user="postgres",
    password="ASkl1289@"
)
cur = conn.cursor()

# Insert into category table
for category in category_list:
    cur.execute(
        "INSERT INTO sports_radar_schema.category (category_id, category_name) VALUES (%s, %s) ON CONFLICT (category_id) DO NOTHING",
        (category['category_id'], category['category_name'])
    )

# Insert into competition table
for comp in com_list:
    cur.execute(        
        "INSERT INTO sports_radar_schema.competition (competition_id, competition_name, type, gender, category_id,parent_id) VALUES (%s, %s, %s, %s, %s,%s)",
        (
            comp['competition_id'],
            comp['competition_name'],
            comp['type'],
            comp['gender'],
            comp['category_id'],
            comp['parent_id']
        )
    )


# Commit and close
conn.commit()
cur.close()
conn.close()

print("Data inserted successfully.")


Data inserted successfully.
